In [1]:
# imports

import os
import json
from dotenv import load_dotenv
from openai import OpenAI
import gradio as gr

In [2]:
load_dotenv() 
openai_api_key = os.getenv('OPENAI_API_KEY') 


MODEL = 'gpt-4o-mini'
openai = OpenAI()

In [3]:
system_message = """You are a helpful assistant for an airline called FlightAI
Give short, courteous answers, no more than 1 sentence. Always be accurate. If you don't 
know the answer say so. 
"""

In [5]:
def chat(message, history):
    messages = (
        [{"role": "system", "content": system_message}]
        + history
        + [{"role": "user", "content": message}]
    )

    response = openai.chat.completions.create(model = MODEL, messages=messages) 
    return response.choices[0].message.content 

gr.ChatInterface(fn = chat, type='messages').launch(inline=False)

* Running on local URL:  http://127.0.0.1:7860

To create a public link, set `share=True` in `launch()`.


In [7]:
# Let's start by making a useful function

ticket_prices = {"london": "$799", "paris": "$899", "tokyo": "$1400", "berlin": "$499"}

def get_ticket_price(destination_city): 
    print(f"Tool get_ticket_price called for {destination_city}") 
    city = destination_city.lower() 
    return ticket_prices.get(city,"Unknown")

In [8]:
get_ticket_price('Berlin')

Tool get_ticket_price called for Berlin


'$499'

In [9]:
price_function = {
    "name": "get_ticket_price",
    "description": "Get the price of a return ticket to the destination city. Call this whenever you need to know the ticket price, for example when a customer asks 'How much is a ticket to this city'",
    "parameters": {
        "type": "object",
        "properties": {
            "destination_city": {
                "type": "string",
                "description": "The city that the customer wants to travel to",
            },
        },
        "required": ["destination_city"],
        "additionalProperties": False,
    },
}

In [11]:
tools= [{'type':'function','function':price_function}]

In [42]:
def handle_tool_call(message):
    tool_call = message.tool_calls[0]

    arguments = json.loads(tool_call.function.arguments)
    city = arguments.get("destination_city")

    price = get_ticket_price(city)

    response = {
        "role": "tool",
        "content": json.dumps({"destination_city": city, "price": price}),
        "tool_call_id": tool_call.id,
    }

    return response, city

In [43]:
def chat(message, history):
    messages = [{"role": "system", "content": system_message}] + history + [{"role": "user", "content": message}]
    response = openai.chat.completions.create(model=MODEL, messages=messages, tools=tools)

    if response.choices[0].finish_reason=="tool_calls":
        message = response.choices[0].message
        response, city = handle_tool_call(message)
        messages.append(message)
        messages.append(response)
        response = openai.chat.completions.create(model=MODEL, messages=messages)
    
    return response.choices[0].message.content